In [1]:
import os
import random
import time
import json
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from pycocotools.coco import COCO
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2

import matplotlib.pyplot as plt
from torch.cuda.amp import GradScaler, autocast
from sklearn.model_selection import StratifiedKFold
import segmentation_models_pytorch as smp

from utils import *
from dataloader import *
from loss import *
from evaluate import *
from scheduler import *

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print (f"This notebook use {device}")

In [2]:
SEED = 77
BATCH_SIZE = 8
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)
random.seed(SEED)

## Train Function
사용하는 train 코드로 바꿔주시면 됩니다.

In [3]:
def fold_train(model, train_loader, val_loader, EPOCHS=21, save_model_name='fold_default'):
    
    # hyper parameters
    LR_start = 2e-6
    LR_max = 1e-4
    accumulation_step = 1
    print_every = 1
    best_val_mIoU = 0.42
    best_val_mIoU2 = 0.44
    best_val_mIoU3 = 0.52

    criterion = IoU_CE_Loss(iou_rate=0.4, weight=None)
    optimizer = torch.optim.Adam(model.parameters(), lr=LR_start)
    scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0=20, eta_max=LR_max, T_up=2, gamma=0.5)

    for epoch in range(EPOCHS):
        epoch+=1
        avg_loss = 0
        batch_count = len(train_loader)

        for step, (images, masks) in enumerate(train_loader):
            start = time.time()
            imgs, masks = images.to(device), masks.long().to(device)

            output = model(imgs)
            loss = criterion(output, masks)
            loss.backward()

            if (step+1)%accumulation_step==0:
                optimizer.step()
                optimizer.zero_grad()

            avg_loss += loss.item() / batch_count
            print(f"\rEpoch:{epoch:3d}  step:{step:3d}/{batch_count-1}  time:{time.time() - start:.3f}  LR:{scheduler.get_lr()[0]:.6f}", end='')

        scheduler.step()
        val_loss, val_mIoU, val_mIoU2, val_mIoU3 = validation3(model, val_loader, criterion, device)
        print(f"   loss:{avg_loss:.3f}  val_loss:{val_loss:.3f}  val_mIoU:{val_mIoU:.3f}  val_mIoU2:{val_mIoU2:.3f}  val_mIooU3:{val_mIoU3:.3f}  {epoch}")
        if best_val_mIoU < val_mIoU:
            save_model(model, saved_dir="model", file_name=save_model_name + f'_epoch{epoch}_miou1.pt')
            best_val_mIoU = val_mIoU
        elif best_val_mIoU2 < val_mIoU2:
            save_model(model, saved_dir="model", file_name=save_model_name + f'_epoch{epoch}_miou2.pt')
            best_val_mIoU2 = val_mIoU2
        elif best_val_mIoU3 < val_mIoU3:
            save_model(model, saved_dir="model", file_name=save_model_name + f'_epoch{epoch}_miou3.pt')
            best_val_mIoU3 = val_mIoU3

## Dataset 클래스

In [ ]:
class PseudoKFoldDataset(Dataset):
    """COCO format"""
    def __init__(self, dataset, transform = None):
        super().__init__()
        self.dataset = dataset
        self.transform = transform
        self.coco = COCO('input/data/train_all.json')
        self.dataset_path = 'input/data/'
        self.category_names = ['Backgroud', 'UNKNOWN', 'General trash', 'Paper', 'Paper pack', 'Metal', 'Glass', 'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing']
        
    def __getitem__(self, index: int):
        
        ### load image ###
        image_infos = self.dataset[index]
        images = cv2.imread(self.dataset_path+image_infos['file_name'])
        images = cv2.cvtColor(images, cv2.COLOR_BGR2RGB).astype(np.float32)
        images /= 255.0
        ### Pseudo mask ###
        if image_infos['pseudo']:
            masks = np.load(self.dataset_path+image_infos['mask_path'])
            
        ### Train mask ###
        else:
            ann_ids = self.coco.getAnnIds(imgIds=image_infos['id'])
            anns = self.coco.loadAnns(ann_ids)
            cat_ids = self.coco.getCatIds()
            cats = self.coco.loadCats(cat_ids)
            
            masks = np.zeros((image_infos["height"], image_infos["width"]))
            for i in range(len(anns)):
                className = get_classname(anns[i]['category_id'], cats)
                pixel_value = self.category_names.index(className)
                masks = np.maximum(self.coco.annToMask(anns[i])*pixel_value, masks)            
        masks = masks.astype(np.float32)

        ###  augmentation ###
        if self.transform is not None:
            transformed = self.transform(image=images, mask=masks)
            images = transformed["image"]
            masks = transformed["mask"]
        return images, masks
    
    def __len__(self):
        return len(self.dataset)

## data load & transform

In [4]:
kfold_dataset = np.load('input/data/pseudo_kfold_all.npy', allow_pickle=True)
anns_cnt = np.load('input/data/pseudo_kfold_anns.npy')

train_transform = A.Compose([
    A.Resize(256, 256),
    A.RandomRotate90(),
    A.HorizontalFlip(p=0.5),
    ToTensorV2()
])

test_transform = A.Compose([
    A.Resize(256, 256),
    ToTensorV2()
])

## 5-Fold train

In [ ]:
skf = StratifiedKFold(n_splits=5)
for k, (train_idx, valid_idx) in enumerate(skf.split(kfold_dataset, anns_cnt)):
    
    ## DataLoader ##
    train_dataset = PseudoKFoldDataset(dataset=kfold_dataset[train_idx], transform=train_transform)
    val_dataset = KFoldDataset(dataset=kfold_dataset[valid_idx], transform=train_transform)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=1, drop_last=True)
    val_loader = torch.utils.data.DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=1)
    
    ## model ##
    model = smp.DeepLabV3Plus(
        encoder_name='resnext50_32x4d',
        encoder_weights='swsl',
        classes=12
    ).to(device)
    
    ## train ##
    print("-"*50 + f" Fold{k+1} Start training " + "-"*50)
    fold_train(model, train_loader, val_loader, EPOCHS=21, save_model_name=f'[fold{k+1}]rxt50_resize_rotateFlip')      ## train 함수 수정 필요 ##
    print("-"*50 + f" Fold{k+1} Finish training " + "-"*50)

loading annotations into memory...
Done (t=4.03s)
creating index...
index created!
loading annotations into memory...
Done (t=4.82s)
creating index...
index created!
-------------------------------------------------- Fold1 Start training --------------------------------------------------
Epoch:  1  step:368/402  time:0.274  LR:0.000002